# Autoencoder

Source: https://www.youtube.com/watch?v=LjRvMUk59PI
https://blog.keras.io/building-autoencoders-in-keras.html

In [183]:
import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Deezer Dataset

In [190]:
data = pd.read_csv('../Data/train_short.csv').iloc[:, [0, 2, 3, 4, 6, 7, 9, 10, 11, 13, 14]]

d1 = data.groupby('user_id')['is_listened'].sum()
d2 = d1[d1 > 10].index
d3 = data[data.user_id.isin(d2)].iloc[:, [0, 3, 4, 5, 6, 7, 9]]

#df_r = pd.pivot_table(d3, index=['user_id'], columns=['media_id'], values=['is_listened'], fill_value=0).astype(int)

print(d3.info())
cutoff = 600
x_train_NN = d3.iloc[:cutoff, :].to_numpy()
x_test_NN = d3.iloc[cutoff:,:].to_numpy()
print(x_train_NN.shape)
print(x_test_NN.shape)
print(x_test_NN)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 9 to 4956
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   genre_id         752 non-null    int64
 1   context_type     752 non-null    int64
 2   platform_name    752 non-null    int64
 3   platform_family  752 non-null    int64
 4   listen_type      752 non-null    int64
 5   user_gender      752 non-null    int64
 6   user_age         752 non-null    int64
dtypes: int64(7)
memory usage: 47.0 KB
None
(600, 7)
(152, 7)
[[ 7  3  0 ...  0  0 28]
 [ 7  3  0 ...  0  0 28]
 [ 7  3  0 ...  0  0 28]
 ...
 [ 7  0  0 ...  0  0 20]
 [ 7  0  0 ...  0  0 20]
 [ 7  1  2 ...  1  0 22]]


## Creating the Architecture of an Autoencoder

In [191]:
# This is the size of our encoded representations
encoding_dim = 4  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(x_train_NN.shape[1],))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu',activity_regularizer=regularizers.l1(10e-5))(input_img) # ,activity_regularizer=regularizers.l1(10e-5)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(x_train_NN.shape[1], activation='relu',kernel_regularizer=l2(0.0001))(encoded)

# This model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
autoencoder.summary()
# This model maps an input to its encoded representation
encoder = Model(input_img, encoded)
# This is our encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss='MSE', metrics=['accuracy', 'mse'])

Model: "model_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 7)]               0         
                                                                 
 dense_32 (Dense)            (None, 4)                 32        
                                                                 
 dense_33 (Dense)            (None, 7)                 35        
                                                                 
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


In [193]:
batch_size = 64
autoencoder.fit(x_train_NN, x_train_NN,
                epochs=100,
                batch_size= batch_size,
                shuffle=True,
                validation_data=(x_test_NN, x_test_NN),
                steps_per_epoch=x_train_NN.shape[0] // batch_size + 1,
                validation_steps=x_test_NN.shape[0] // batch_size + 1)


Epoch 1/100
10/10 [==============================] - 0s 6ms/step - loss: 134455399220710385647616.0000 - accuracy: 0.7517 - mse: 403689.5312 - val_loss: 134426747319881054552064.0000 - val_accuracy: 0.6842 - val_mse: 3776498.0000
Epoch 2/100
10/10 [==============================] - 0s 3ms/step - loss: 134405048976876383502336.0000 - accuracy: 0.7517 - mse: 403689.5625 - val_loss: 134374748758583434805248.0000 - val_accuracy: 0.6842 - val_mse: 3776498.0000
Epoch 3/100
10/10 [==============================] - 0s 3ms/step - loss: 134352464947627205591040.0000 - accuracy: 0.7517 - mse: 403689.5312 - val_loss: 134321624297378972434432.0000 - val_accuracy: 0.6842 - val_mse: 3776498.5000
Epoch 4/100
10/10 [==============================] - 0s 3ms/step - loss: 134299151335238393659392.0000 - accuracy: 0.7517 - mse: 403689.5625 - val_loss: 134268112526606556200960.0000 - val_accuracy: 0.6842 - val_mse: 3776498.5000
Epoch 5/100
10/10 [==============================] - 0s 3ms/step - loss: 1342456

In [194]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test_NN)
decoded_imgs = decoder.predict(encoded_imgs)

In [195]:
np.set_printoptions(suppress=True)
print(np.around(decoded_imgs, 0))

[[ 0.  4.  0. ...  0.  0. 26.]
 [ 0.  4.  0. ...  0.  0. 26.]
 [ 0.  4.  0. ...  0.  0. 26.]
 ...
 [ 0.  4.  0. ...  0.  0. 26.]
 [ 0.  4.  0. ...  0.  0. 26.]
 [ 0.  4.  0. ...  0.  0. 26.]]


In [196]:
df = pd.DataFrame(decoded_imgs)
df.tail()

,0,1,2,3,4,5,6
147,0.0,3.514018,0.0,0.0,0.0,0.0,25.864637
148,0.0,3.514018,0.0,0.0,0.0,0.0,25.864637
149,0.0,3.514018,0.0,0.0,0.0,0.0,25.864637
150,0.0,3.514018,0.0,0.0,0.0,0.0,25.864637
151,0.0,3.514018,0.0,0.0,0.0,0.0,25.864637
